# First and Second Derivative of Function

In [ ]:
using ITensors
using Plots
using HDF5

Define function of interest

In [ ]:
N = 10; # Number of sites
s = siteinds("Qubit", N); # Indices for all sites
xs = range(0, 2pi*(1-(1/2^N)), length=2^N); # Sampling grid points
step_size = 2pi/(2^N);

f(x) = cos(x);
ys = f.(xs);

plot(xs, ys) # Plotting the function

Encode function as MPS

In [ ]:
cutoff = 1E-16;
χ = 30;

ψ = MPS(ys, s, cutoff=cutoff, maxdim=χ);

orthogonalize!(ψ,1)

@show linkdims(ψ);

Define MPO for first and second derivatives, using central finite difference up to second order

In [ ]:
# Left shift:

left_ls = zeros(2, 2, 2) 
middle_ls = zeros(2, 2, 2, 2) 
right_ls = zeros(2, 2, 2) 

left_ls[1, 2, 2] = 1
left_ls[2, 1, 1] = 1
middle_ls[1, 1, 1, 1] = 1
middle_ls[1, 2, 2, 1] = 1
middle_ls[2, 2, 1, 1] = 1
middle_ls[2, 1, 2, 2] = 1
right_ls[1, 1, 1] = 1
right_ls[1, 2, 2] = 1
right_ls[2, 1, 2] = 1
right_ls[2, 2, 1] = 1

H_ls = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_ls[i] = ITensor(left_ls, s[i], s[i]', α[i]);
    elseif i == N # last site
        H_ls[i] = ITensor(right_ls, α[i-1], s[i], s[i]');
    else # middle site
        H_ls[i] = ITensor(middle_ls, α[i-1], s[i], s[i]', α[i]);
    end
end

orthogonalize!(H_ls,1);

# Right shift:

left_rs = zeros(2, 2, 2) 
middle_rs = zeros(2, 2, 2, 2) 
right_rs = zeros(2, 2, 2) 

left_rs[1, 2, 1] = 1
left_rs[2, 1, 2] = 1
middle_rs[1, 1, 1, 1] = 1
middle_rs[1, 2, 2, 1] = 1
middle_rs[2, 1, 2, 1] = 1
middle_rs[2, 2, 1, 2] = 1
right_rs[1, 1, 1] = 1
right_rs[1, 2, 2] = 1
right_rs[2, 1, 2] = 1
right_rs[2, 2, 1] = 1

H_rs = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_rs[i] = ITensor(left_rs, s[i], s[i]', α[i]);
    elseif i == N # last site
        H_rs[i] = ITensor(right_rs, α[i-1], s[i], s[i]');
    else # middle site
        H_rs[i] = ITensor(middle_rs, α[i-1], s[i], s[i]', α[i]);
    end
end

orthogonalize!(H_rs,1);

# Identity:

left_id = zeros(2, 2, 1) 
middle_id = zeros(1, 2, 2, 1) 
right_id = zeros(1, 2, 2) 

left_id[1, 1, 1] = 1
left_id[2, 2, 1] = 1
middle_id[1, 1, 1, 1] = 1
middle_id[1, 2, 2, 1] = 1
right_id[1, 1, 1] = 1
right_id[1, 2, 2] = 1

H_id = MPO(N);

α = [Index(1) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_id[i] = ITensor(left_id, s[i], s[i]', α[i]);
    elseif i == N # last site
        H_id[i] = ITensor(right_id, α[i-1], s[i], s[i]');
    else # middle site
        H_id[i] = ITensor(middle_id, α[i-1], s[i], s[i]', α[i]);
    end
end

orthogonalize!(H_id,1);

# Define derivatives
H_dudx = 0.5*(H_ls - H_rs)/step_size; # First derivative
H_d2udx2 = (H_rs + H_ls - 2*H_id)/(step_size^2); # Second derivative

Apply derivatives to MPS encoding function

In [ ]:
Deriv1 = apply(H_dudx,ψ);
Deriv2 = apply(H_d2udx2,ψ);

@show linkdims(Deriv1);
@show linkdims(Deriv2);

Contract the resulting MPS and plot the results

In [ ]:
# Recovering the original big tensors
Big_ψ1 = contract(Deriv1); 
Big_ψ2 = contract(Deriv2); 

# Changing the format from ITensor to Julia array for plotting
J_recon1 = Array(Big_ψ1, s) 
J_recon2 = Array(Big_ψ2, s) 

# Reshaping from N-dimentional tensor to regular array
ys_recon1 = reshape(J_recon1,2^N,1) 
ys_recon2 = reshape(J_recon2,2^N,1) 

# Plotting the reconstructed derivatives of the function
plot(xs, ys_recon1) 
plot!(xs, ys_recon2) 

Exercise finished!!!